In [28]:
import os
import openai
import sys

sys.path.append("../..")

# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())  # read local .env file

# openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = "sk-xx"

In [37]:
os.environ["OPENAI_API_KEY"] = "sk-xx"


In [15]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [16]:
# What does the loaded documents look like?
print(f"type of doc: {type(documents)}", "\n")
print(f"the number of docs: {len(documents)}", "\n")
print(f"the type of the first doc: {type(documents[3])}")
print(f"The first doc itself: {documents[3]}")

type of doc: <class 'list'> 

the number of docs: 41 

the type of the first doc: <class 'llama_index.schema.Document'>
The first doc itself: Doc ID: 2a3527c1-4b95-4983-8cd8-50e881711870
Text: PAGE 4Coding AI Is the New Literacy Today we take it for granted
that many people know how to read and write. Someday, I hope,  it will
be just as common that people know how to write code, specifically for
AI. Several hundred years ago, society didn’t view language literacy
as a necessary skill. A small  number of people learned to read and
wri...


In [17]:
# The document is now parsed as 41 separate pages.  If we merge them together it will be easier to work with.
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [18]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [19]:
query_engine = index.as_query_engine()

In [29]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

When finding projects to build your experience, there are several steps you can take. First, you can join existing projects by asking to join someone else's project if they have an idea. Additionally, you can continue reading, taking courses, and talking to domain experts to come up with new ideas. It is also helpful to focus on a specific application area where machine learning has not yet been applied. This can give you the opportunity to explore unique and creative applications that no one else has done yet. Finally, you can develop a side hustle or personal project that may not initially be part of your job but can still stir your creative juices and strengthen bonds with collaborators.


## Evaluation setup using TruLens

In [21]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find part of AI challenging?


In [22]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [23]:
print(len(eval_questions))

11


In [26]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [38]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [39]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [40]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [41]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_d1bedbca2d1254d321d470ac381c67b2,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_d1bedbca2d1254d321d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:32.093018"", ""...",2023-12-27T04:05:36.251264,1.0,0.95,1.00,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",4,2185,0.003361
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e8e8d8fca5a9e3f23ac1a9d803048323,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_e8e8d8fca5a9e3f23ac...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:36.586749"", ""...",2023-12-27T04:05:39.391579,1.0,0.00,1.00,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,2,1712,0.002612
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_723b03c80f5c5a9ac400e690a7613aa7,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_723b03c80f5c5a9ac40...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:39.775468"", ""...",2023-12-27T04:05:42.654252,1.0,0.00,0.25,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,2,1706,0.002599
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_f97c42708dbb5dc64556dd83f95d3462,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_f97c42708dbb5dc6455...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:42.990384"", ""...",2023-12-27T04:05:45.096983,1.0,0.50,1.00,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,2,1676,0.002554
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_94470e64650f839b2de9274390e6afc2,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_94470e64650f839b2de...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:05:45.431139"", ""...",2023-12-27T04:05:47.821476,1.0,0.00,0.50,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,2,1668,0.002539


In [42]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


ModuleNotFoundError: No module named 'ipywidgets'

: 